In [1]:
import pandas as pd
import numpy as np

In [80]:
s = pd.Series(['my cat', 'he is fat', 'railway station'])
s.str[::-1].str.match('ta[f|g]|n')

0    False
1     True
2     True
dtype: bool

In [106]:
s = pd.Series(['上海市黄浦区方浜中路249号',
               '上海市宝山区密山路5号',
               '北京市昌平区北农路2号'])
pat = '(?P<one>\w+) (?P<two>\w+) (?P<three>\w+)'
pat = '(\w+市)(\w+区)(\w+路)(\d+号)'
city = {'上海市': 'Shanghai', '北京市': 'Beijing'}
district = {'昌平区': 'CP District',
            '黄浦区': 'HP District',
            '宝山区': 'BS District'}
road = {'方浜中路': 'Mid Fangbin Road',
        '密山路': 'Mishan Road',
        '北农路': 'Beinong Road'}
def my_func(m):
    str_city = city[m.group(1)]
    str_district = district[m.group(2)]
    str_road = road[m.group(3)]
    str_no = 'No. ' + m.group(4)[:-1]
    return ' '.join([str_city,
                     str_district,
                     str_road,
                     str_no])
s.str.replace(pat, my_func)

0    Shanghai HP District Mid Fangbin Road No. 249
1           Shanghai BS District Mishan Road No. 5
2           Beijing CP District Beinong Road No. 2
dtype: object

In [61]:
s = pd.Series(['A135T15,A26S5','B674S2,B25T6'], index = ['my_A','my_B'])
pat = '[A|B](\d+)[T|S](\d+)'
s.str.extractall(pat)
pat_with_name = '[A|B](?P<name1>\d+)[T|S](?P<name2>\d+)'
s.str.findall(pat_with_name)

my_A    [(135, 15), (26, 5)]
my_B     [(674, 2), (25, 6)]
dtype: object

In [69]:
s = pd.Series(['a_b','c_d'])
s.str.translate({'a':'a_new', 'c':'c_new'})

0    a_b
1    c_d
dtype: object

In [107]:
pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'
def my_func(m):
    str_city = city[m.group('市名')]
    str_district = district[m.group('区名')]
    str_road = road[m.group('路名')]
    str_no = 'No. ' + m.group('编号')[:-1]
    return ' '.join([str_city,
                     str_district,
                     str_road,
                     str_no])
s.str.replace(pat, my_func)

0    Shanghai HP District Mid Fangbin Road No. 249
1           Shanghai BS District Mishan Road No. 5
2           Beijing CP District Beinong Road No. 2
dtype: object

In [110]:
pat = '(\w+市)(\w+区)(\w+路)(\d+号)'
s.str.extract(pat)

,0,1,2,3
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


In [89]:
s = pd.Series(np.random.randint(1,10000,100000)).astype('string')
#s.astype('string').str.zfill(6)

In [91]:
%timeit s.str.zfill(6)

28.5 ms ± 583 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [98]:
s = pd.Series(['my cat', 'he is fat', 'railway station'])
s.str.contains('^[m|h]')
s.str.contains('([f|g]at|n)$')

0    False
1     True
2     True
dtype: bool

In [83]:
s.str.ljust(5,'*')

0    a****
1    b****
2    c****
dtype: object

In [84]:
s.str.rjust(5,'*')

0    ****a
1    ****b
2    ****c
dtype: object

In [85]:
s.str.zfill(5)

0    0000a
1    0000b
2    0000c
dtype: object

In [11]:
s = pd.Series(['cat rat fat at', 'get feed sheet heat', 'at ee fat feed', 'good, feet'])
s.str.findall('[r|f]at|ee').astype('string').str.extractall("'(\w+)'").unstack().droplevel(0, axis=1)

match,0,1,2
0,rat,fat,<NA>
1,ee,ee,<NA>
2,ee,fat,ee
3,ee,<NA>,<NA>


In [13]:
s = pd.Series(['cat rat fat at',
                   'get feed sheet heat',
                   'at ee fat feed',
                   'good, feet'])
s.str.findall('[r|f]at|ee')

0       [rat, fat]
1         [ee, ee]
2    [ee, fat, ee]
3             [ee]
dtype: object

In [56]:
s2.index = [1, 2]
s1.str.cat(s2, sep='-', na_rep='?', join='outer')

0      a-?
1    b-cat
2    ?-dog
dtype: object

In [135]:
s = pd.Series(['b'], dtype='object')
s.str[0]

0    b
dtype: object

In [136]:
s.str.isdecimal()

0    False
dtype: bool